# Маркетинговый анализ
***

# Описание проекта

Вы — маркетинговый аналитик развлекательного приложения Procrastinate Pro+. Несмотря на огромные вложения в рекламу, последние несколько месяцев компания терпит убытки. Ваша задача — разобраться в причинах и помочь компании выйти в плюс.
Есть данные о пользователях, привлечённых с 1 мая по 27 октября 2019 года:

- лог сервера с данными об их посещениях,
- выгрузка их покупок за этот период,
- рекламные расходы.

Вам предстоит изучить:

- откуда приходят пользователи и какими устройствами они пользуются,
- сколько стоит привлечение пользователей из различных рекламных каналов;
- сколько денег приносит каждый клиент,
- когда расходы на привлечение клиента окупаются,
- какие факторы мешают привлечению клиентов.

## Оглавление<a class="anchor" id="TOC"></a>
***

* **[I. Загрузка и подготовка данных](#1)**
    - [1) Импортируем библиотеки, объявим класс MetricCalculator для анализа](#1_1)
    - [2) Прочитаем данные](#1_2)
    - [3) Выведем первые и последние пять строк каждой из таблиц, общую информацию](#1_3)
    - [5) Выведем общую статистку по датасетам и наличие дубликатов](#1_5)
    - [Вывод](#1_conclusion)
* **[II. Обучение и проверка модели](#2)**
    - [1. Инициализируем модель линейной регрессии и создадим словарь для поиска наилучших параметров](#2_model)
    - [2. Соберем отчет для каждого из регионов](#2_regions)
    - [3. Результаты моделей для каждого из регионов](#2_model_results)
    - [4. Сравнение правильного среднего и среднего, предсказанного моделью](#2_mean)
    - [Вывод](#2_conclusion)
* **[III. Подготовка к расчёту прибыли](#3)**
    - [1) Выделим ключевые значения для расчетов](#3_1)
    - [2) Подсчитаем минимальный средний объём сырья в месторождениях региона, достаточный для его разработки](#3_2)
    - [3) Подготовим функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели](#3_3)
    - [Вывод](#3_conclusion)
* **[IV. Расчёт прибыли и рисков](#4)**
    - [1) Найдем распределение прибыли с помощью Bootstrap](#4_1)
    - [2) Подсчитаем среднюю прибыль, 95-% доверительный интервал и риск убытков](#4_2)
    - [Вывод](#4_conclusion)

# I. Загрузка и подготовка данных<a class='anchor' id='1'>
***

## 1) Импортируем библиотеки, объявим класс MetricCalculator для анализа<a class="anchor" id="1_1"></a>

In [1]:
# тут импорты
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from calculator.calculator import MetricCalculator
from IPython.display import display

In [2]:
# тут класс

In [3]:
# тут полезное
pd.set_option('display.float_format', lambda x: '%.2f' % x)

[В оглавление](#TOC)

## 2) Прочитаем данные<a class="anchor" id="1_2"></a>

### - описание данных

Файл visits_info_short.csv хранит лог сервера с информацией о посещениях сайта, orders_info_short.csv — информацию о покупках, а costs_info_short.csv — информацию о расходах на рекламу.


Структура visits_info_short.csv|Описание|Структура orders_info_short.csv|Описание|Структура costs_info_short.csv|Описание
:-----|:-----|:-----|:-----|:-----|:-----  
User Id|уникальный идентификатор пользователя|User Id|уникальный идентификатор пользователя|Channel|идентификатор рекламного источника
Region|страна пользователя|Event Dt|дата и время покупки|Dt|дата проведения рекламной кампании
Device|тип устройства пользователя|Revenue|сумма заказа|Costs|расходы на эту кампанию
Channel|идентификатор источника перехода| | | | 
Session Start|дата и время начала сессии| | | | 
Session End|дата и время окончания сессии| | | | 

### - создадим экземпляр класса с путями до данных 

In [4]:
calc = MetricCalculator('datasets/visits_info_short.csv', 'datasets/orders_info_short.csv', 'datasets/costs_info_short.csv')

In [5]:
calc.columns_fixer()

Cоздан экземпляр класса:
- данные прочитаны, 
- наименование колонок приведено к единому формату,
- колонки с датами приведены к соответствующему формату.

Датасеты доступны через экземпляр класса по именам **visits, orders, costs**.

## 3) Выведем первые пять первых и последних строк каждой из таблиц, общую информацию<a class="anchor" id="1_3"></a>

### - visits - посещение сайта

In [6]:
calc.visits

,user_id,region,device,channel,session_start,session_end
0,981449118918,United States,iPhone,organic,2019-05-01 02:36:01,2019-05-01 02:45:01
1,278965908054,United States,iPhone,organic,2019-05-01 04:46:31,2019-05-01 04:47:35
2,590706206550,United States,Mac,organic,2019-05-01 14:09:25,2019-05-01 15:32:08
3,326433527971,United States,Android,TipTop,2019-05-01 00:29:59,2019-05-01 00:54:25
4,349773784594,United States,Mac,organic,2019-05-01 03:33:35,2019-05-01 03:57:40
...,...,...,...,...,...,...
309896,329994900775,UK,PC,LeapBob,2019-10-31 13:28:12,2019-10-31 14:39:29
309897,334903592310,France,PC,lambdaMediaAds,2019-10-31 22:14:52,2019-10-31 22:39:36
309898,540102010126,Germany,PC,organic,2019-10-31 01:40:48,2019-10-31 01:41:31
309899,308736936846,Germany,Mac,organic,2019-10-31 07:37:34,2019-10-31 07:37:55


In [7]:
calc.visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        309901 non-null  int64         
 1   region         309901 non-null  object        
 2   device         309901 non-null  object        
 3   channel        309901 non-null  object        
 4   session_start  309901 non-null  datetime64[ns]
 5   session_end    309901 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 14.2+ MB


&#9889; **Вывод:** в датасете с информацией о посещениях сайта 309901 строка, без пропусков, без проблем с типами данных.

### - orders - информация о покупках

In [8]:
calc.orders

,user_id,event_dt,revenue
0,188246423999,2019-05-01 23:09:52,4.99
1,174361394180,2019-05-01 12:24:04,4.99
2,529610067795,2019-05-01 11:34:04,4.99
3,319939546352,2019-05-01 15:34:40,4.99
4,366000285810,2019-05-01 13:59:51,4.99
...,...,...,...
40207,651604369137,2019-10-31 16:19:07,4.99
40208,275341387049,2019-10-31 01:17:17,4.99
40209,374656616484,2019-10-31 06:17:29,4.99
40210,168548862926,2019-10-31 22:46:19,4.99


In [9]:
calc.orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   40212 non-null  int64         
 1   event_dt  40212 non-null  datetime64[ns]
 2   revenue   40212 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 942.6 KB


&#9889; **Вывод:** в датасете с информацией о покупках 40212 строк (13% от числа визитов), без пропусков, без проблем с типами данных.

### - costs - информация о расходах

In [10]:
calc.costs

,dt,channel,costs
0,2019-05-01,FaceBoom,113.30
1,2019-05-02,FaceBoom,78.10
2,2019-05-03,FaceBoom,85.80
3,2019-05-04,FaceBoom,136.40
4,2019-05-05,FaceBoom,122.10
...,...,...,...
1795,2019-10-23,lambdaMediaAds,4.00
1796,2019-10-24,lambdaMediaAds,6.40
1797,2019-10-25,lambdaMediaAds,8.80
1798,2019-10-26,lambdaMediaAds,8.80


In [11]:
calc.costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


&#9889; **Вывод:** в датасете с информацией расходах 1800, без пропусков, без проблем с типами данных.

[В оглавление](#TOC)

## 5) Выведем общую статистку по датасетам, проверим наличие дубликатов и уникальные значения<a class="anchor" id="1_5"></a>

**В отношении user_id считать статистику неинформативно - поэтому исключим данный столбец**

### - visits - посещение сайта

In [12]:
calc.visits.describe(exclude=['int64']).T.fillna('---')

,count,unique,top,freq,first,last
region,309901,4,United States,207327,---,---
device,309901,4,iPhone,112603,---,---
channel,309901,11,organic,107760,---,---
session_start,309901,306813,2019-06-27 14:56:41,3,2019-05-01 00:00:41,2019-10-31 23:59:23
session_end,309901,306793,2019-07-16 02:23:30,3,2019-05-01 00:07:06,2019-11-01 01:38:46


В датасете о посещениях сайта:

- сведения о 4 регионах, больше всего пользователей из США - 207327 из 309901 наблюдения (67%),
- сведения о 4 устройствах,больше всего iPhone - 112603 из 309901 наблюдения (36%),
- более трети пользователей пришли напрямую - 107760 из 309901 (35%),
- самое раннее наблюдение о первом действии - 01.05.2019 года, самое позднее - 31.10.2019 года.

### - orders - информация о покупках

In [13]:
calc.orders.describe(include=['float64']).T

,count,mean,std,min,25%,50%,75%,max
revenue,40212.00,5.37,3.45,4.99,4.99,4.99,4.99,49.99


In [14]:
calc.orders.describe(include=['object', 'datetime64[ns]']).T

,count,unique,top,freq,first,last
event_dt,40212,40163,2019-10-26 06:49:22,2,2019-05-01 00:28:11,2019-10-31 23:56:56


С учетом того, что 75%-процентиль - 4.99 (совпадающий с медианой, 25%-процентилем и минимуом), максимальная сумма выручки - 49.99, проверим какие значения выручки есть в датасете:

In [41]:
sorted(calc.orders.revenue.unique())

[4.99, 5.99, 9.99, 19.99, 49.99]

Всего пять значений стоимости услуги. Проверим сколько пользователей платят больше 4.99, возможно, таких пользователей стоит рассматривать отдельно:

In [54]:
max_revenue = calc.orders.groupby('revenue').agg({'user_id': 'count'}).reset_index()
max_revenue['%'] = max_revenue['user_id'] / len(calc.orders)
max_revenue

,revenue,user_id,%
0,4.99,38631,0.96
1,5.99,780,0.02
2,9.99,385,0.01
3,19.99,204,0.01
4,49.99,212,0.01


Примерно 5% пользователей платят больше базовой стоимости. Посчитаем общее число таких пользователей:

In [62]:
sum(max_revenue[max_revenue['revenue'] > 4.99]['user_id'])

1581

В датасете с информацией о покупках:

- остутствуют сведения о валюте в которой выражена выручка,
- принимая во внимание, что 67% пользователей из США, с разумной степенью достоверности можно предположить, что валюта выручки - доллары США,
- выявлено 5 различных стоимостей услуг - 4.99, 5.99, 9.99, 19.99, 49.99.
- минимальная сумма выручки - 4.99, средняя - 5.37 (при медианной и 75%-процентиле - 4.99), максимальная - 49.99,
- Из 40185 платящих пользователей - 1581 (5%) платит больше базового тарифа,
- самое раннее наблюдение о первом действии совпадает с датасетом о посещениях - 01.05.2019 года, самое позднее - 31.10.2019 года.

### - costs - информация о расходах

In [15]:
calc.costs.describe().T

,count,mean,std,min,25%,50%,75%,max
costs,1800.00,58.61,107.74,0.80,6.50,12.29,33.60,630.00


In [16]:
calc.costs.describe(include=['object', 'datetime64[ns]']).T

,count,unique,top,freq
dt,1800,180,2019-05-01,10
channel,1800,10,FaceBoom,180


С учетом характера данных - общая статистика малоинформативна без точного понимания, какие именно атрибуты наблюдений содержаться в столбцах f0, f1, f2.

### - проверим наличие полных дубликатов

In [65]:
(calc.visits.duplicated() == True).sum()

0

In [64]:
(calc.orders.duplicated() == True).sum()

0

In [63]:
(calc.costs.duplicated() == True).sum()

0

Полных дубликатов нет

&#9889; **Вывод:**
1. Описательная статистика неинформативна без точного описания данных без точного понимания, какие именно атрибуты наблюдений содержаться в столбцах f0, f1, f2.
2. В датасетах нет полных дубликатов.
3. В каждом из датасетов в столбце id есть повторяющиеся значения: df_geo_1 - 10 из 100 000 (0,0001‬%), df_geo_2 - 4 из 100 000 (0,00004%), df_geo_3 - 4 из 100 000 (0,00004%). Вероятно, из некторых скважин брали несколько замеров, скорее всего на разных глубинах.
4. Исходя из описания данных, столбец id кандидат на удаление, поскольку уникальный идентификатор месторождения - не универсальный признак.

[В оглавление](#TOC)

## Вывод<a class="anchor" id="1_conclusion"></a>

Исходя из представленных данных и описаний данных, можно сделать следующие выводы:
1. В датасетах представлены данные георазведки из трех регионов по 100 000 уникальных наблюдений в каждом.
2. Во всех датасетах нет проблем с типами данных, полные дубликаты отсутствуют, нет пропусков.
3. Есть повторяющиеся значения в столбцах с id: df_geo_1 - 10 из 100 000 (0,0001‬%), df_geo_2 - 4 из 100 000 (0,00004%), df_geo_3 - 4 из 100 000 (0,00004%). 
4. Поскольку нет возможности уточнить у источника данных - исходим из буквального описания датасета: есть наблюдения из одних тех же скважин. 
5. Столбец id не будет использован для обучения модели, поскольку является неуниверсальным атрибутом.

[В оглавление](#TOC)

In [ ]:
from calculator.calculator import MetricCalculator

In [ ]:
calc = MetricCalculator('datasets/visits_info_short.csv', 'datasets/orders_info_short.csv', 'datasets/costs_info_short.csv')

In [ ]:
calc.columns_fixer()

In [ ]:
observation_date = datetime(2019, 11, 1).date()
horizon_days = 30

In [ ]:
profiles = calc.get_profiles()

In [ ]:
profiles

In [ ]:
retention_raw, retention_grouped, retention_hist = calc.get_retention(profiles, observation_date, horizon_days)

In [ ]:
calc.plot_retention(retention_grouped, retention_hist, horizon_days)

In [ ]:
cr_raw, cr_grouped, cr_hist = calc.get_conversion(profiles, observation_date, horizon_days)

In [ ]:
calc.plot_conversion(cr_grouped, cr_hist, horizon_days)

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days)

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['region'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days, window=14) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['channel'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 

In [ ]:
ltv_raw, ltv_grouped, ltv_hist, roi_grouped, roi_hist = calc.get_ltv(profiles, observation_date, horizon_days, dimensions=['payer'])

In [ ]:
calc.plot_ltv_roi(ltv_grouped, ltv_hist, roi_grouped, roi_hist, horizon_days) 

In [ ]:
profiles

In [ ]:
roi_hist

In [ ]:
cac_hist = roi_hist[['cac']]

In [ ]:
columns = [name for name in cac_hist.index.names if name not in ['dt']]

In [ ]:
filtered_data = cac_hist.pivot_table(index='dt', columns=columns, values='cac', aggfunc='mean')

In [ ]:
filtered_data

In [ ]:
def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 

In [ ]:
filtered_data.plot()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
filter_data(filtered_data, 14).plot()
plt.axhline(y=1, color='red', linestyle='--',
            label='Уровень окупаемости')

In [ ]:
roi_grouped

In [ ]:
from datetime import datetime